In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
# from art.estimators.classification import PyTorchClassifier

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./TRADES.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./TRADES.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense_Label/WUSTL_Def3/y_pred_{attack_name}{eps}_Def3.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')


In [11]:
def squared_l2_norm(x):
    flattened = x.view(x.unsqueeze(0).shape[0], -1)
    return (flattened ** 2).sum(1)


def l2_norm(x):
    return squared_l2_norm(x).sqrt()

def trades_loss(model,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=1.0,
                distance='l_inf'):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(size_average=False)
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()
    if distance == 'l_inf':
        for _ in range(perturb_steps):
            x_adv.requires_grad_()
            with torch.enable_grad():
                loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(model(x_natural), dim=1))
            grad = torch.autograd.grad(loss_kl, [x_adv])[0]
            x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
            x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
            x_adv = torch.clamp(x_adv, 0.0, 1.0)
    elif distance == 'l_2':
        delta = 0.001 * torch.randn(x_natural.shape).cuda().detach()
        delta = Variable(delta.data, requires_grad=True)

        # Setup optimizers
        optimizer_delta = optim.SGD([delta], lr=epsilon / perturb_steps * 2)

        for _ in range(perturb_steps):
            adv = x_natural + delta

            # optimize
            optimizer_delta.zero_grad()
            with torch.enable_grad():
                loss = (-1) * criterion_kl(F.log_softmax(model(adv), dim=1),
                                           F.softmax(model(x_natural), dim=1))
            loss.backward()
            # renorming gradient
            grad_norms = delta.grad.view(batch_size, -1).norm(p=2, dim=1)
            delta.grad.div_(grad_norms.view(-1, 1, 1, 1))
            # avoid nan or inf if gradient is 0
            if (grad_norms == 0).any():
                delta.grad[grad_norms == 0] = torch.randn_like(delta.grad[grad_norms == 0])
            optimizer_delta.step()

            # projection
            delta.data.add_(x_natural)
            delta.data.clamp_(0, 1).sub_(x_natural)
            delta.data.renorm_(p=2, dim=0, maxnorm=epsilon)
        x_adv = Variable(x_natural + delta, requires_grad=False)
    else:
        x_adv = torch.clamp(x_adv, 0.0, 1.0)
    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    # calculate robust loss
    logits = model(x_natural)
    loss_natural = F.cross_entropy(logits, y)
    loss_robust = (1.0 / batch_size) * criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                                    F.softmax(model(x_natural), dim=1))
    loss = loss_natural + beta * loss_robust
    return loss

In [12]:
# # Adversarial training loop
# for epoch in range(10):
#     model.train()
#     train_loss = 0.0
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)

#         optimizer.zero_grad()
#         # calculate robust loss - TRADES loss
#         loss = trades_loss(model=model,
#                            x_natural=data,
#                            y=target,
#                            optimizer=optimizer,
#                            step_size=0.01,
#                            epsilon=0.3,
#                            perturb_steps=10,
#                            beta=1.0,
# 			   distance='l_inf')
#         loss.backward()
#         optimizer.step()
#         # print progress
#         if batch_idx % 1000 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
            
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break


In [13]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/TRADES/TRADES.pt"))

/tmp/ipykernel_10815/2847205294.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense

<All keys matched successfully>

In [14]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [15]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./TRADES.csv", mode='a', index=False, header=False)

In [ ]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
(235303, 42) (235303,)
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 FGSM
(28948, 42) (28948,)
Accuracy: 0.9801713417161807

macro
Precision: 0.6624791234340723
Recall: 0.8022826989447281
F1 Score: 0.6637108535909064

weighted
Precision: 0.9866175574971076
Recall: 0.9801713417161807
F1 Score: 0.9813985233595338

start XGB 100 PGD
(235303, 42) (235303,)
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 DF
(6282, 42) (6282,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8955746577523082

macro
Precision: 0.20229685271794962
Recall: 0.3796485623003195
F1 Score: 0.194034386050976

weighted
Precision: 0.9957953855496725
Recall: 0.8955746577523082
F1 Score: 0.9427738298405285

start XGB 100 AutoPGD
(397616, 42) (397616,)
Accuracy: 0.9873219387549796

macro
Precision: 0.7231505499896829
Recall: 0.7081187491090929
F1 Score: 0.7061185940177696

weighted
Precision: 0.9871488429773219
Recall: 0.9873219387549796
F1 Score: 0.9869746617422885

start XGB 100 ZOO
(20, 42) (20,)
Accuracy: 0.5

macro
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

weighted
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

start XGB 100 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(3731, 42) (3731,)
Accuracy: 0.8549986598767086

macro
Precision: 0.5312950997724225
Recall: 0.7877720682844993
F1 Score: 0.5384433025763529

weighted
Precision: 0.899951178455317
Recall: 0.8549986598767086
F1 Score: 0.8759949670291249

start XGB 100 SINIFGSM
(338681, 42) (338681,)
Accuracy: 0.9840971297474614

macro
Precision: 0.5652979497785832
Recall: 0.6681998983306198
F1 Score: 0.5844041539894859

weighted
Precision: 0.9835979539195295
Recall: 0.9840971297474614
F1 Score: 0.9835837128636312

start XGB 100 VNIFGSM
(75620, 42) (75620,)
Accuracy: 0.9913250462840518

macro
Precision: 0.9141383200561762
Recall: 0.8582704569200768
F1 Score: 0.8758515828744432

weighted
Precision: 0.9915615934196683
Recall: 0.9913250462840518
F1 Score: 0.9907209410237885

start XGB 50 baseline
(7, 42) (7,)
Accuracy: 0.8571428571428571

macro
Precision: 0.42857142857142855
Recall: 0.5
F1 Score: 0.46153846153846156

weighted
Precision: 0.7346938775510203
Recall: 0.8571428571428571
F1 Score: 0.7912087912087

...

(487070, 42) (487070,)
Accuracy: 0.9904551707146817

macro
Precision: 0.7261956579052512
Recall: 0.6860508551762761
F1 Score: 0.682527207181613

weighted
Precision: 0.9901606783398906
Recall: 0.9904551707146817
F1 Score: 0.9899788785840827

start RF 50 FGSM
(244269, 42) (244269,)
Accuracy: 0.9958242756960564

macro
Precision: 0.6831441213329088
Recall: 0.8576385520461087
F1 Score: 0.7076122304456202

weighted
Precision: 0.9964983074509562
Recall: 0.9958242756960564
F1 Score: 0.9959559230802971

start RF 50 PGD
(487070, 42) (487070,)
Accuracy: 0.9904551707146817

macro
Precision: 0.7261956579052512
Recall: 0.6860508551762761
F1 Score: 0.682527207181613

weighted
Precision: 0.9901606783398906
Recall: 0.9904551707146817
F1 Score: 0.9899788785840827

start RF 50 DF
(6060, 42) (6060,)
Accuracy: 0.8887788778877888

macro
Precision: 0.3152903445108588
Recall: 0.7492173830803424
F1 Score: 0.3338961259381874

weighted
Precision: 0.9971308277205145
Recall: 0.8887788778877888
F1 Score: 0.93898930

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

(2761, 42) (2761,)
Accuracy: 0.8500543281419776

macro
Precision: 0.5137306067673207
Recall: 0.7846782963718543
F1 Score: 0.5241874249180574

weighted
Precision: 0.9339517588882418
Recall: 0.8500543281419776
F1 Score: 0.8867577045985279

start RF 50 SINIFGSM
(648733, 42) (648733,)
Accuracy: 0.9904829876081531

macro
Precision: 0.5884921570090696
Recall: 0.696098068454632
F1 Score: 0.5933940873523355

weighted
Precision: 0.9907651272087736
Recall: 0.9904829876081531
F1 Score: 0.9903005492084807

start RF 50 VNIFGSM
(258764, 42) (258764,)
Accuracy: 0.9950920529903696

macro
Precision: 0.8860034306946274
Recall: 0.7933017387928913
F1 Score: 0.8208635755955515

weighted
Precision: 0.9952182235848888
Recall: 0.9950920529903696
F1 Score: 0.9947064321301365

start RF 20 baseline
(8, 42) (8,)
Accuracy: 0.625

macro
Precision: 0.2777777777777778
Recall: 0.3333333333333333
F1 Score: 0.30303030303030304

weighted
Precision: 0.5208333333333334
Recall: 0.625
F1 Score: 0.5681818181818181

start RF 2

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

(484264, 42) (484264,)


In [ ]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
(7, 42) (7,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8571428571428571

macro
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1 Score: 0.3333333333333333

weighted
Precision: 0.8571428571428571
Recall: 0.8571428571428571
F1 Score: 0.8571428571428571

start XGB 100 BIM
(289904, 42) (289904,)
Accuracy: 0.9950707820519896

macro
Precision: 0.7801185557002928
Recall: 0.8390819899283901
F1 Score: 0.7996347354110638

weighted
Precision: 0.9960542723105723
Recall: 0.9950707820519896
F1 Score: 0.9953919072334012

start XGB 100 FGSM
(244664, 42) (244664,)
Accuracy: 0.997752019095576

macro
Precision: 0.6434510279811044
Recall: 0.802442037907763
F1 Score: 0.6564938967562544

weighted
Precision: 0.9985166220304618
Recall: 0.997752019095576
F1 Score: 0.99802636487835

start XGB 100 PGD
(289904, 42) (289904,)
Accuracy: 0.9950707820519896

macro
Precision: 0.7801185557002928
Recall: 0.8390819899283901
F1 Score: 0.7996347354110638

weighted
Precision: 0.9960542723105723
Recall: 0.9950707820519896
F1 Score: 0.9953919072334012

start

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

(243731, 42) (243731,)
Accuracy: 0.3035026320000328

macro
Precision: 0.24993126402715332
Recall: 0.7127361117914033
F1 Score: 0.17995300201010958

weighted
Precision: 0.9229099360591464
Recall: 0.3035026320000328
F1 Score: 0.40786080191854407

start XGB 100 SINIFGSM
(665813, 42) (665813,)
Accuracy: 0.9947432687556416

macro
Precision: 0.6875828476522753
Recall: 0.762694764174084
F1 Score: 0.7115669697669109

weighted
Precision: 0.9950897810521204
Recall: 0.9947432687556416
F1 Score: 0.9948939328839721

start XGB 100 VNIFGSM
(242288, 42) (242288,)
Accuracy: 0.9989764247507099

macro
Precision: 0.9096703997878184
Recall: 0.8481991305396693
F1 Score: 0.872994218850247

weighted
Precision: 0.9989791708443913
Recall: 0.9989764247507099
F1 Score: 0.9989500755295514

start XGB 50 baseline
(18, 42) (18,)
Accuracy: 0.5555555555555556

macro
Precision: 0.4
Recall: 0.5
F1 Score: 0.4375

weighted
Precision: 0.3555555555555555
Recall: 0.5555555555555556
F1 Score: 0.4305555555555556

start XGB 50 B

...

(488053, 42) (488053,)
Accuracy: 0.9908145221932864

macro
Precision: 0.7345982335306507
Recall: 0.683970688505204
F1 Score: 0.6854773567659762

weighted
Precision: 0.990379118012282
Recall: 0.9908145221932864
F1 Score: 0.9903494527459451

start RF 10 FGSM
(243366, 42) (243366,)
Accuracy: 0.9978345372812965

macro
Precision: 0.7022171081730265
Recall: 0.8506505221690521
F1 Score: 0.7378812814910999

weighted
Precision: 0.9984419826547146
Recall: 0.9978345372812965
F1 Score: 0.9980834113936334

start RF 10 PGD
(488053, 42) (488053,)
Accuracy: 0.9908145221932864

macro
Precision: 0.7345982335306507
Recall: 0.683970688505204
F1 Score: 0.6854773567659762

weighted
Precision: 0.990379118012282
Recall: 0.9908145221932864
F1 Score: 0.9903494527459451

start RF 10 DF
(6131, 42) (6131,)
Accuracy: 0.889414451149894

macro
Precision: 0.3378432326588704
Recall: 0.7627583591429113
F1 Score: 0.38898984217590693

weighted
Precision: 0.9935065909388109
Recall: 0.889414451149894
F1 Score: 0.93716241048

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(522638, 42) (522638,)
Accuracy: 0.2376654586922497

macro
Precision: 0.24001174395164596
Recall: 0.6838318277599972
F1 Score: 0.14838598123512306

weighted
Precision: 0.9086248015286461
Recall: 0.2376654586922497
F1 Score: 0.31395502396771174

start RF 10 SINIFGSM
(709614, 42) (709614,)
Accuracy: 0.9908725025154521

macro
Precision: 0.5741942247140406
Recall: 0.6955904071555423
F1 Score: 0.5742263805796348

weighted
Precision: 0.9915821118886728
Recall: 0.9908725025154521
F1 Score: 0.9909208466322944

start RF 10 VNIFGSM
(256280, 42) (256280,)
Accuracy: 0.996695021070704

macro
Precision: 0.9408380109414856
Recall: 0.8029485909018786
F1 Score: 0.8562661621016365

weighted
Precision: 0.9967042303810606
Recall: 0.996695021070704
F1 Score: 0.9964638118845557

start RF 1 baseline
(8, 42) (8,)
Accuracy: 0.875

macro
Precision: 0.4375
Recall: 0.5
F1 Score: 0.4666666666666667

weighted
Precision: 0.765625
Recall: 0.875
F1 Score: 0.8166666666666667

start RF 1 BIM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(490981, 42) (490981,)
Accuracy: 0.9899996130196485

macro
Precision: 0.6889801062784194
Recall: 0.6202571534277995
F1 Score: 0.5921802345201403

weighted
Precision: 0.9894927644973681
Recall: 0.9899996130196485
F1 Score: 0.9893403180103607

start RF 1 FGSM
(237871, 42) (237871,)


In [ ]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



In [22]:
epsilon = 0

Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.5555555555555556

macro
Precision: 0.20833333333333334
Recall: 0.3333333333333333
F1 Score: 0.25641025641025644

weighted
Precision: 0.3472222222222222
Recall: 0.5555555555555556
F1 Score: 0.4273504273504274

start BIM
Accuracy: 0.9939177083494685

macro
Precision: 0.7058046628047387
Recall: 0.5496258696820595
F1 Score: 0.5757375103972541

weighted
Precision: 0.9938252723038068
Recall: 0.9939177083494685
F1 Score: 0.9923740453603375

start FGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8437686344663089

macro
Precision: 0.43459102680303374
Recall: 0.6273732430032276
F1 Score: 0.45979061245219854

weighted
Precision: 0.8641880037184467
Recall: 0.8437686344663089
F1 Score: 0.8517006461054438

start PGD
Accuracy: 0.9939177083494685

macro
Precision: 0.7058046628047387
Recall: 0.5496258696820595
F1 Score: 0.5757375103972541

weighted
Precision: 0.9938252723038068
Recall: 0.9939177083494685
F1 Score: 0.9923740453603375

start DF


...

Accuracy: 0.6

macro
Precision: 0.25
Recall: 0.3333333333333333
F1 Score: 0.2857142857142857

weighted
Precision: 0.45
Recall: 0.6
F1 Score: 0.5142857142857142

start CaFA
Accuracy: 0.2652190606202807

macro
Precision: 0.2820664742981486
Recall: 0.686264706482198
F1 Score: 0.22355536810920124

weighted
Precision: 0.7115355676258963
Recall: 0.2652190606202807
F1 Score: 0.23461418857093697

start SINIFGSM
Accuracy: 0.9600922389565468

macro
Precision: 0.4490131126629738
Recall: 0.6474181912903274
F1 Score: 0.4877512250946353

weighted
Precision: 0.9726189930284047
Recall: 0.9600922389565468
F1 Score: 0.9655872526543323

start VNIFGSM
Accuracy: 0.9872579001019368

macro
Precision: 0.9272506532958816
Recall: 0.7338571685372222
F1 Score: 0.7786204720735863

weighted
Precision: 0.9866944329915265
Recall: 0.9872579001019368
F1 Score: 0.9859474707472614

start baseline


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.7142857142857143

macro
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1 Score: 0.3333333333333333

weighted
Precision: 0.7142857142857143
Recall: 0.7142857142857143
F1 Score: 0.7142857142857143

start BIM
Accuracy: 0.9930152272055397

macro
Precision: 0.4574061744368896
Recall: 0.4234440082075392
F1 Score: 0.43801535395498165

weighted
Precision: 0.9923147139658773
Recall: 0.9930152272055397
F1 Score: 0.9925943586886588

start FGSM
Accuracy: 0.8551582197067147

macro
Precision: 0.4344315372896495
Recall: 0.5244163304457725
F1 Score: 0.43034519879809396

weighted
Precision: 0.8680706764539062
Recall: 0.8551582197067147
F1 Score: 0.8609234324991939

start PGD
Accuracy: 0.9930152272055397

macro
Precision: 0.4574061744368896
Recall: 0.4234440082075392
F1 Score: 0.43801535395498165

weighted
Precision: 0.9923147139658773
Recall: 0.9930152272055397
F1 Score: 0.9925943586886588

start DF
Accuracy: 0.8869910282953761

macro
Precision: 0.2463874168273771
Recall: 0.52549

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.7894736842105263

macro
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1 Score: 0.3333333333333333

weighted
Precision: 0.7894736842105263
Recall: 0.7894736842105263
F1 Score: 0.7894736842105263

start CaFA
Accuracy: 0.23416706750040084

macro
Precision: 0.2736732630876979
Recall: 0.7721693408338817
F1 Score: 0.19492218064763359

weighted
Precision: 0.8351359831432981
Recall: 0.23416706750040084
F1 Score: 0.2484656699648847

start SINIFGSM
Accuracy: 0.9899268605497591

macro
Precision: 0.57052468677913
Recall: 0.5316930044583585
F1 Score: 0.5466015891094547

weighted
Precision: 0.9861893329362845
Recall: 0.9899268605497591
F1 Score: 0.9879732682527816

start VNIFGSM
Accuracy: 0.9954992742912688

macro
Precision: 0.8005190899428865
Recall: 0.7322519924712387
F1 Score: 0.7606213187488

weighted
Precision: 0.9956302496279864
Recall: 0.9954992742912688
F1 Score: 0.9951953231544638



In [17]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
(235303, 42) (235303,)
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 FGSM
(28948, 42) (28948,)
Accuracy: 0.9801713417161807

macro
Precision: 0.6624791234340723
Recall: 0.8022826989447281
F1 Score: 0.6637108535909064

weighted
Precision: 0.9866175574971076
Recall: 0.9801713417161807
F1 Score: 0.9813985233595338

start XGB 100 PGD
(235303, 42) (235303,)
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 DF
(6282, 42) (6282,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8955746577523082

macro
Precision: 0.20229685271794962
Recall: 0.3796485623003195
F1 Score: 0.194034386050976

weighted
Precision: 0.9957953855496725
Recall: 0.8955746577523082
F1 Score: 0.9427738298405285

start XGB 100 AutoPGD
(397616, 42) (397616,)
Accuracy: 0.9873219387549796

macro
Precision: 0.7231505499896829
Recall: 0.7081187491090929
F1 Score: 0.7061185940177696

weighted
Precision: 0.9871488429773219
Recall: 0.9873219387549796
F1 Score: 0.9869746617422885

start XGB 100 ZOO
(20, 42) (20,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5

macro
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

weighted
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

start XGB 100 CaFA
(3731, 42) (3731,)
Accuracy: 0.8549986598767086

macro
Precision: 0.5312950997724225
Recall: 0.7877720682844993
F1 Score: 0.5384433025763529

weighted
Precision: 0.899951178455317
Recall: 0.8549986598767086
F1 Score: 0.8759949670291249

start XGB 100 SINIFGSM
(338681, 42) (338681,)
Accuracy: 0.9840971297474614

macro
Precision: 0.5652979497785832
Recall: 0.6681998983306198
F1 Score: 0.5844041539894859

weighted
Precision: 0.9835979539195295
Recall: 0.9840971297474614
F1 Score: 0.9835837128636312

start XGB 100 VNIFGSM
(75620, 42) (75620,)
Accuracy: 0.9913250462840518

macro
Precision: 0.9141383200561762
Recall: 0.8582704569200768
F1 Score: 0.8758515828744432

weighted
Precision: 0.9915615934196683
Recall: 0.9913250462840518
F1 Score: 0.9907209410237885

start XGB 50 baseline
(14, 42) (14,)
Accuracy: 0.6428571428571429

macr

...

(2832, 42) (2832,)
Accuracy: 0.8788841807909604

macro
Precision: 0.5031852885117641
Recall: 0.7704781758739487
F1 Score: 0.5119740638830526

weighted
Precision: 0.949316192339949
Recall: 0.8788841807909604
F1 Score: 0.9105057207398535

start RF 100 SINIFGSM
(711488, 42) (711488,)
Accuracy: 0.9908543784294324

macro
Precision: 0.573987138363569
Recall: 0.6959084506503058
F1 Score: 0.5741739637174503

weighted
Precision: 0.9915533354534831
Recall: 0.9908543784294324
F1 Score: 0.9908961721825019

start RF 100 VNIFGSM
(259848, 42) (259848,)
Accuracy: 0.9933576552446045

macro
Precision: 0.8436929355353439
Recall: 0.8154134965658221
F1 Score: 0.8208236994623785

weighted
Precision: 0.9936370564301533
Recall: 0.9933576552446045
F1 Score: 0.9927912771277748

start RF 50 baseline
(18, 42) (18,)
Accuracy: 0.5

macro
Precision: 0.1875
Recall: 0.3333333333333333
F1 Score: 0.24

weighted
Precision: 0.28125
Recall: 0.5
F1 Score: 0.36

start RF 50 BIM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

(274310, 42) (274310,)
Accuracy: 0.9869709452808866

macro
Precision: 0.7931205707272606
Recall: 0.7356667933625657
F1 Score: 0.7559806018574358

weighted
Precision: 0.9865823200956984
Recall: 0.9869709452808866
F1 Score: 0.98667630282512

start RF 50 FGSM
(243532, 42) (243532,)
Accuracy: 0.9959553569962059

macro
Precision: 0.7091468238671845
Recall: 0.8189877362382717
F1 Score: 0.7277759816703339

weighted
Precision: 0.9964197196489463
Recall: 0.9959553569962059
F1 Score: 0.9960053318940694

start RF 50 PGD
(274310, 42) (274310,)
Accuracy: 0.9869709452808866

macro
Precision: 0.7931205707272606
Recall: 0.7356667933625657
F1 Score: 0.7559806018574358

weighted
Precision: 0.9865823200956984
Recall: 0.9869709452808866
F1 Score: 0.98667630282512

start RF 50 DF
(7051, 42) (7051,)
Accuracy: 0.8013047794639059

macro
Precision: 0.232975780989599
Recall: 0.6489577579213635
F1 Score: 0.2352631355401041

weighted
Precision: 0.9939330944917414
Recall: 0.8013047794639059
F1 Score: 0.88526340921

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(4188, 42) (4188,)
Accuracy: 0.8512416427889207

macro
Precision: 0.45700275852787636
Recall: 0.7150832662426876
F1 Score: 0.4355581373027905

weighted
Precision: 0.9365936964079855
Recall: 0.8512416427889207
F1 Score: 0.890664520529451

start RF 50 SINIFGSM
(616255, 42) (616255,)
Accuracy: 0.9900268557658761

macro
Precision: 0.5774290213457418
Recall: 0.6689967872082996
F1 Score: 0.5774257097717653

weighted
Precision: 0.9902957685634106
Recall: 0.9900268557658761
F1 Score: 0.9898268158062858

start RF 50 VNIFGSM
(256792, 42) (256792,)
Accuracy: 0.9939445154054644

macro
Precision: 0.9420580172169604
Recall: 0.7822736944198585
F1 Score: 0.8435178504821756

weighted
Precision: 0.9941116822496225
Recall: 0.9939445154054644
F1 Score: 0.9933306542009341

start RF 20 baseline
(2901, 42) (2901,)
Accuracy: 0.9968976215098242

macro
Precision: 0.9260671878577738
Recall: 0.8875
F1 Score: 0.8764534620027492

weighted
Precision: 0.9977874641105068
Recall: 0.9968976215098242
F1 Score: 0.99656327

In [18]:
epsilon = 0
Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
(235303, 42) (235303,)
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 FGSM
(28948, 42) (28948,)
Accuracy: 0.9801713417161807

macro
Precision: 0.6624791234340723
Recall: 0.8022826989447281
F1 Score: 0.6637108535909064

weighted
Precision: 0.9866175574971076
Recall: 0.9801713417161807
F1 Score: 0.9813985233595338

start XGB 100 PGD
(235303, 42) (235303,)
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 DF
(6282, 42) (6282,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8955746577523082

macro
Precision: 0.20229685271794962
Recall: 0.3796485623003195
F1 Score: 0.194034386050976

weighted
Precision: 0.9957953855496725
Recall: 0.8955746577523082
F1 Score: 0.9427738298405285

start XGB 100 AutoPGD
(397616, 42) (397616,)
Accuracy: 0.9873219387549796

macro
Precision: 0.7231505499896829
Recall: 0.7081187491090929
F1 Score: 0.7061185940177696

weighted
Precision: 0.9871488429773219
Recall: 0.9873219387549796
F1 Score: 0.9869746617422885

start XGB 100 ZOO
(20, 42) (20,)
Accuracy: 0.5

macro
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

weighted
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

start XGB 100 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(3731, 42) (3731,)
Accuracy: 0.8549986598767086

macro
Precision: 0.5312950997724225
Recall: 0.7877720682844993
F1 Score: 0.5384433025763529

weighted
Precision: 0.899951178455317
Recall: 0.8549986598767086
F1 Score: 0.8759949670291249

start XGB 100 SINIFGSM
(338681, 42) (338681,)
Accuracy: 0.9840971297474614

macro
Precision: 0.5652979497785832
Recall: 0.6681998983306198
F1 Score: 0.5844041539894859

weighted
Precision: 0.9835979539195295
Recall: 0.9840971297474614
F1 Score: 0.9835837128636312

start XGB 100 VNIFGSM
(75620, 42) (75620,)
Accuracy: 0.9913250462840518

macro
Precision: 0.9141383200561762
Recall: 0.8582704569200768
F1 Score: 0.8758515828744432

weighted
Precision: 0.9915615934196683
Recall: 0.9913250462840518
F1 Score: 0.9907209410237885

start XGB 50 baseline
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 BIM
(278343, 42) (278343,)
Accuracy: 0.9890243332866284

macro
Precision: 

...

Accuracy: 0.9632248939179632

macro
Precision: 0.3616470751497935
Recall: 0.592657953097721
F1 Score: 0.3773344641274116

weighted
Precision: 0.9995544075867295
Recall: 0.9632248939179632
F1 Score: 0.9809732674126035

start RF 10 AutoPGD
(538548, 42) (538548,)
Accuracy: 0.8948301729836523

macro
Precision: 0.5987003954586478
Recall: 0.729547424803805
F1 Score: 0.6117558538718553

weighted
Precision: 0.9488644222093386
Recall: 0.8948301729836523
F1 Score: 0.910700792862717

start RF 10 ZOO
(22, 42) (22,)
Accuracy: 0.9545454545454546

macro
Precision: 0.4772727272727273
Recall: 0.5
F1 Score: 0.4883720930232558

weighted
Precision: 0.9111570247933884
Recall: 0.9545454545454546
F1 Score: 0.9323467230443974

start RF 10 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(7523, 42) (7523,)
Accuracy: 0.5774292170676592

macro
Precision: 0.5759855526067948
Recall: 0.8044137960723152
F1 Score: 0.5257678016667069

weighted
Precision: 0.8798902051116257
Recall: 0.5774292170676592
F1 Score: 0.6620394539819879

start RF 10 SINIFGSM
(712268, 42) (712268,)
Accuracy: 0.990868605637204

macro
Precision: 0.5744250503680027
Recall: 0.6995421929096539
F1 Score: 0.5748747530694936

weighted
Precision: 0.9915609665368418
Recall: 0.990868605637204
F1 Score: 0.9909073563445885

start RF 10 VNIFGSM
(274275, 42) (274275,)
Accuracy: 0.9943851973384377

macro
Precision: 0.8137161261095927
Recall: 0.8019282493894051
F1 Score: 0.7998201464589413

weighted
Precision: 0.9944614479736057
Recall: 0.9943851973384377
F1 Score: 0.9939727992710496

start RF 1 baseline
(4, 42) (4,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 BIM
(624042, 42) (624042,)
Accuracy: 0.9949827094971172

macro
Precision: 0.68511

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9628416791740574

macro
Precision: 0.40410012206430457
Recall: 0.5772224770722517
F1 Score: 0.39632384803297943

weighted
Precision: 0.9994955801485422
Recall: 0.9628416791740574
F1 Score: 0.9807594222163032

start RF 1 AutoPGD
(465542, 42) (465542,)
Accuracy: 0.8814134922305613

macro
Precision: 0.521383104843875
Recall: 0.6623813308249922
F1 Score: 0.5216748145709932

weighted
Precision: 0.9627947340935997
Recall: 0.8814134922305613
F1 Score: 0.9094974580939931

start RF 1 ZOO
(14, 42) (14,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 CaFA
(24377, 42) (24377,)
Accuracy: 0.3163637855355458

macro
Precision: 0.3755781603364527
Recall: 0.679727362211203
F1 Score: 0.32310743684381593

weighted
Precision: 0.8249508159936381
Recall: 0.3163637855355458
F1 Score: 0.3432702681284906

start RF 1 SINIFGSM
(572252, 42) (572252,)
Accuracy: 0.9926990207111552

macro
Precision: 0.5265329328794336
Recall: 0.

In [19]:
epsilon = 0

Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]
active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 FGSM
Accuracy: 0.9801713417161807

macro
Precision: 0.6624791234340723
Recall: 0.8022826989447281
F1 Score: 0.6637108535909064

weighted
Precision: 0.9866175574971076
Recall: 0.9801713417161807
F1 Score: 0.9813985233595338

start XGB 100 PGD
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8955746577523082

macro
Precision: 0.20229685271794962
Recall: 0.3796485623003195
F1 Score: 0.194034386050976

weighted
Precision: 0.9957953855496725
Recall: 0.8955746577523082
F1 Score: 0.9427738298405285

start XGB 100 AutoPGD
Accuracy: 0.9873219387549796

macro
Precision: 0.7231505499896829
Recall: 0.7081187491090929
F1 Score: 0.7061185940177696

weighted
Precision: 0.9871488429773219
Recall: 0.9873219387549796
F1 Score: 0.9869746617422885

start XGB 100 ZOO


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5

macro
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

weighted
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

start XGB 100 CaFA
Accuracy: 0.8549986598767086

macro
Precision: 0.5312950997724225
Recall: 0.7877720682844993
F1 Score: 0.5384433025763529

weighted
Precision: 0.899951178455317
Recall: 0.8549986598767086
F1 Score: 0.8759949670291249

start XGB 100 SINIFGSM
Accuracy: 0.9840971297474614

macro
Precision: 0.5652979497785832
Recall: 0.6681998983306198
F1 Score: 0.5844041539894859

weighted
Precision: 0.9835979539195295
Recall: 0.9840971297474614
F1 Score: 0.9835837128636312

start XGB 100 VNIFGSM
Accuracy: 0.9913250462840518

macro
Precision: 0.9141383200561762
Recall: 0.8582704569200768
F1 Score: 0.8758515828744432

weighted
Precision: 0.9915615934196683
Recall: 0.9913250462840518
F1 Score: 0.9907209410237885

start XGB 50 baseline
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

...

Accuracy: 0.907919280990334

macro
Precision: 0.2017857142857143
Recall: 0.3815744825246013
F1 Score: 0.19388219739746673

weighted
Precision: 0.9994958090070011
Recall: 0.907919280990334
F1 Score: 0.951236708593807

start RF 20 AutoPGD
Accuracy: 0.9915304554663822

macro
Precision: 0.7078882206743223
Recall: 0.7482805587644096
F1 Score: 0.7099388971209961

weighted
Precision: 0.9914782930917772
Recall: 0.9915304554663822
F1 Score: 0.9914131877858501

start RF 20 ZOO
Accuracy: 0.875

macro
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1 Score: 0.3333333333333333

weighted
Precision: 0.875
Recall: 0.875
F1 Score: 0.875

start RF 20 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9091226843760922

macro
Precision: 0.48727316349010535
Recall: 0.8555587741951454
F1 Score: 0.4969909648950585

weighted
Precision: 0.9748479321080152
Recall: 0.9091226843760922
F1 Score: 0.9389483380350881

start RF 20 SINIFGSM
Accuracy: 0.9923310545717302

macro
Precision: 0.5723658878624927
Recall: 0.7127954256799205
F1 Score: 0.5872717913077652

weighted
Precision: 0.9931712041371072
Recall: 0.9923310545717302
F1 Score: 0.9926252102674639

start RF 20 VNIFGSM
Accuracy: 0.9994136964818192

macro
Precision: 0.9038594517227209
Recall: 0.8639552499662051
F1 Score: 0.8818436264087566

weighted
Precision: 0.9994046510863128
Recall: 0.9994136964818192
F1 Score: 0.999407238538784

start RF 10 baseline
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 BIM
Accuracy: 0.9929101472309625

macro
Precision: 0.7018082953326953
Recall: 0.7467027545148934
F1 Score: 0.683808395660608

weighted
Precision: 0.9929135

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9046148846278843

macro
Precision: 0.201980198019802
Recall: 0.38091367257356523
F1 Score: 0.19390022846526575

weighted
Precision: 0.9995173387992032
Recall: 0.9046148846278843
F1 Score: 0.9494397963668191

start RF 10 AutoPGD
Accuracy: 0.9930909987669544

macro
Precision: 0.7132566142100902
Recall: 0.7695014020160258
F1 Score: 0.709652142001001

weighted
Precision: 0.9932880749222077
Recall: 0.9930909987669544
F1 Score: 0.9930905839905778

start RF 10 ZOO
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 CaFA
Accuracy: 0.5563897763578275

macro
Precision: 0.5888793505575964
Recall: 0.8042693572548497
F1 Score: 0.5063473291800773

weighted
Precision: 0.8898204092608176
Recall: 0.5563897763578275
F1 Score: 0.6227433873933661

start RF 10 SINIFGSM
Accuracy: 0.9926416755215351

macro
Precision: 0.5797423995908028
Recall: 0.7235547524984319
F1 Score: 0.5973621264245099

weighted
Precision: 0.9935448687

In [ ]:
epsilon = 0

Def = "Def3"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"
total_num_attack = ["EOAL", "EOAL_ExcludeCaFA", "EOAL_Exclude2Attack", "EOAL_Exclude3Attack"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{num_attack}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 BIM
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 FGSM
Accuracy: 0.9801713417161807

macro
Precision: 0.6624791234340723
Recall: 0.8022826989447281
F1 Score: 0.6637108535909064

weighted
Precision: 0.9866175574971076
Recall: 0.9801713417161807
F1 Score: 0.9813985233595338

start XGB 100 PGD
Accuracy: 0.9864345120971684

macro
Precision: 0.8165977803771456
Recall: 0.7362273674465538
F1 Score: 0.7685847014376048

weighted
Precision: 0.9855042791489991
Recall: 0.9864345120971684
F1 Score: 0.9858391356834738

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8955746577523082

macro
Precision: 0.20229685271794962
Recall: 0.3796485623003195
F1 Score: 0.194034386050976

weighted
Precision: 0.9957953855496725
Recall: 0.8955746577523082
F1 Score: 0.9427738298405285

start XGB 100 AutoPGD
Accuracy: 0.9873219387549796

macro
Precision: 0.7231505499896829
Recall: 0.7081187491090929
F1 Score: 0.7061185940177696

weighted
Precision: 0.9871488429773219
Recall: 0.9873219387549796
F1 Score: 0.9869746617422885

start XGB 100 ZOO


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5

macro
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

weighted
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

start XGB 100 CaFA
Accuracy: 0.8549986598767086

macro
Precision: 0.5312950997724225
Recall: 0.7877720682844993
F1 Score: 0.5384433025763529

weighted
Precision: 0.899951178455317
Recall: 0.8549986598767086
F1 Score: 0.8759949670291249

start XGB 100 SINIFGSM
Accuracy: 0.9840971297474614

macro
Precision: 0.5652979497785832
Recall: 0.6681998983306198
F1 Score: 0.5844041539894859

weighted
Precision: 0.9835979539195295
Recall: 0.9840971297474614
F1 Score: 0.9835837128636312

start XGB 100 VNIFGSM
Accuracy: 0.9913250462840518

macro
Precision: 0.9141383200561762
Recall: 0.8582704569200768
F1 Score: 0.8758515828744432

weighted
Precision: 0.9915615934196683
Recall: 0.9913250462840518
F1 Score: 0.9907209410237885

start XGB 50 baseline
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

...

Accuracy: 0.8181818181818182

macro
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1 Score: 0.3333333333333333

weighted
Precision: 0.8181818181818182
Recall: 0.8181818181818182
F1 Score: 0.8181818181818182

start RF 20 BIM
Accuracy: 0.9914699077872214

macro
Precision: 0.7700093317478467
Recall: 0.6961960431837236
F1 Score: 0.7126770504947035

weighted
Precision: 0.9907766591391549
Recall: 0.9914699077872214
F1 Score: 0.9908195158769619

start RF 20 FGSM
Accuracy: 0.9952289696960058

macro
Precision: 0.6777764345503685
Recall: 0.888479853520383
F1 Score: 0.6953847038538508

weighted
Precision: 0.9959307974391948
Recall: 0.9952289696960058
F1 Score: 0.9952114604345175

start RF 20 PGD
Accuracy: 0.9914699077872214

macro
Precision: 0.7700093317478467
Recall: 0.6961960431837236
F1 Score: 0.7126770504947035

weighted
Precision: 0.9907766591391549
Recall: 0.9914699077872214
F1 Score: 0.9908195158769619

start RF 20 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9150033266799734

macro
Precision: 0.20174934403657807
Recall: 0.38302263648468704
F1 Score: 0.19465827264819952

weighted
Precision: 0.9991573891267854
Recall: 0.9150033266799734
F1 Score: 0.9549652901715244

start RF 20 AutoPGD
Accuracy: 0.9900894561321343

macro
Precision: 0.7124398314690923
Recall: 0.7400366409405641
F1 Score: 0.7017610258731026

weighted
Precision: 0.9897533454137012
Recall: 0.9900894561321343
F1 Score: 0.9897109863854476

start RF 20 ZOO


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.8888888888888888

macro
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1 Score: 0.3333333333333333

weighted
Precision: 0.8888888888888888
Recall: 0.8888888888888888
F1 Score: 0.8888888888888888

start RF 20 CaFA
Accuracy: 0.8992275280898876

macro
Precision: 0.508461389275099
Recall: 0.8329510066321731
F1 Score: 0.5190164577125356

weighted
Precision: 0.9707392801811826
Recall: 0.8992275280898876
F1 Score: 0.9320637664938896

start RF 20 SINIFGSM


In [ ]:
print("a")
# torch.save(model.state_dict(), "./TRADES.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")